In [36]:
import os
import json
from glob import glob
import random

import torch
import einops
import pandas as pd
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
from IPython.display import display, Markdown, Latex


import shared.utils as su
from notebooks.eval_care_retrieval import load_data

In [2]:
# Load data
dataset = 'ssv2'
df = load_data(dataset=dataset)
df = df.drop_duplicates(subset=['id', 'text_id']).reset_index(drop=True)
df.shape

Number of rows:  1430
Sample row: 
{
    "id": 69703,
    "label": "moving pen up",
    "template": "Moving [something] up",
    "placeholders": "['pen']",
    "target": 114,
    "chiral_label": 0.0,
    "chiral_triplet_id": "3f20f09b",
    "noun": "['something']",
    "text_id": "3f20f09b_0.0",
    "video_path": "/scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/69703.webm"
}


(1430, 10)

In [5]:
# Sample 20% of the data for closed-source eval
subdf = df.sample(frac=0.2, random_state=42)
subdf.shape

(286, 10)

In [11]:
text_id_to_template = dict(zip(df['text_id'], df['template']))
text_id_to_template

{'3f20f09b_0.0': 'Moving [something] up',
 '12055391_1.0': 'Folding [something]',
 '12055391_0.0': 'Unfolding [something]',
 '3f20f09b_1.0': 'Moving [something] down',
 'b58b2f93_0.0': 'Taking [one of many similar things on the table]',
 '743faf6b_0.0': 'Pushing [something] from right to left',
 '743faf6b_1.0': 'Pushing [something] from left to right',
 '1539a698_1.0': 'Closing [something]',
 'b58b2f93_1.0': 'Putting [something similar to other things that are already on the table]',
 'a1383a77_1.0': 'Moving [something] and [something] away from each other',
 'a1383a77_0.0': 'Moving [something] and [something] closer to each other',
 '81dd0413_0.0': 'Opening [something]',
 '106d2603_0.0': 'Pulling [something] from right to left',
 '584d53c4_0.0': 'Opening [something]',
 '545d0d67_0.0': 'Moving [something] towards the camera',
 'ccff0ef2_0.0': 'Turning the camera right while filming [something]',
 '677ffe97_1.0': 'Closing [something]',
 'c1c62263_0.0': 'Opening [something]',
 '677ffe97_

In [20]:
df

,id,label,template,placeholders,target,chiral_label,chiral_triplet_id,noun,text_id,video_path
0,69703,moving pen up,Moving [something] up,['pen'],114,0.0,3f20f09b,['something'],3f20f09b_0.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
1,217571,folding cloth,Folding [something],['cloth'],46,1.0,12055391,['something'],12055391_1.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
2,24837,folding mat,Folding [something],['mat'],46,1.0,12055391,['something'],12055391_1.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
3,96312,unfolding paper,Unfolding [something],['paper'],82,0.0,12055391,['something'],12055391_0.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
4,29910,moving wallet down,Moving [something] down,['wallet'],112,1.0,3f20f09b,['something'],3f20f09b_1.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
...,...,...,...,...,...,...,...,...,...,...
1425,194086,opening a box,Opening [something],['a box'],115,0.0,677ffe97,['object'],677ffe97_0.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
1426,112252,moving charger down,Moving [something] down,['charger'],112,1.0,3f20f09b,['something'],3f20f09b_1.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
1427,86588,taking spoon,Taking [one of many similar things on the table],['spoon'],53,0.0,b58b2f93,['something'],b58b2f93_0.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...
1428,157360,unfolding a shirt,Unfolding [something],['a shirt'],82,0.0,12055391,['something'],12055391_0.0,/scratch/shared/beegfs/piyush/datasets/SSv2/20...


In [53]:
# Load Gemini wrapper

from utils.gemini_utils import GeminiWrapper

model_key = "gemini-2.5-flash"
vlm = GeminiWrapper(model_key=model_key, fps=1.)
vlm

Loading gemini-2.5-flash with FPS=1.0...........................................  


In [54]:
row

{'id': 42956,
 'label': 'approaching keys with your camera',
 'template': 'Approaching [something] with your camera',
 'placeholders': "['keys']",
 'target': 0,
 'chiral_label': 1.0,
 'chiral_triplet_id': '27f40159',
 'noun': "['something']",
 'text_id': '27f40159_1.0',
 'video_path': '/scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/42956.webm'}

In [55]:
PROMPT_TEMPLATE = """
Watch the given video carefully. Which of the following actions best matches that shown in the video?

(1) %s

(2) %s

Just return the final answer (1) or (2).
"""

In [56]:
row

{'id': 42956,
 'label': 'approaching keys with your camera',
 'template': 'Approaching [something] with your camera',
 'placeholders': "['keys']",
 'target': 0,
 'chiral_label': 1.0,
 'chiral_triplet_id': '27f40159',
 'noun': "['something']",
 'text_id': '27f40159_1.0',
 'video_path': '/scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/42956.webm'}

In [58]:
chiral_ids = subdf.chiral_triplet_id.unique()
verbose = False
answers = {}
chiral_index = 0
for chiral_id in chiral_ids:
    rows = subdf[subdf.chiral_triplet_id == chiral_id]
    chiral_index += 1

    for j in su.log.tqdm_iterator(range(len(rows)), desc=f'Processing videos for chiral ID: {chiral_id} ({chiral_index}/{len(chiral_ids)})'):
        row = rows.iloc[j].to_dict()

        options = [
            text_id_to_template[f'{chiral_id}_0.0'],
            text_id_to_template[f'{chiral_id}_1.0'],
        ]

        # Sort randomly
        np.random.shuffle(options)

        ground_truth = row['template']
        ground_truth_index = options.index(ground_truth)

        if 'opening' in ground_truth.lower() or 'closing' in ground_truth.lower():
            # Replace [something] with noun
            ground_truth = ground_truth.replace('[something]', row['noun'])
            options_ = [o.replace('[something]', row['noun']) for o in options]
        else:
            options_ = options

        prompt = PROMPT_TEMPLATE % (options_[0], options_[1])

        # Generate answer
        try:
            answer = vlm.forward_text_video(prompt, row['video_path'])
            answers[row['video_path']] = dict(
                prompt=prompt,
                options=options_,
                ground_truth_index=ground_truth_index,
                answer=answer,
            )
        except:
            print(f"Failed {j}.")
            continue

        if verbose:
            print(chiral_id, row['chiral_label'], ground_truth_index, options_)
            break

    if verbose:
        display(su.visualize.show_single_video(row['video_path']))
        display(Markdown('---'))
        display(Markdown(prompt))
        display(Markdown('---'))
        display(Markdown(answer))
        break
        print('-' * 120)

len(answers)

Processing videos for chiral ID: 677ffe97 (1/16):   0%|          | 0/16 [00:00<?, ?it/s]

Processing videos for chiral ID: b58b2f93 (2/16):   0%|          | 0/33 [00:00<?, ?it/s]

Processing videos for chiral ID: 584d53c4 (3/16):   0%|          | 0/9 [00:00<?, ?it/s]

Processing videos for chiral ID: 81dd0413 (4/16):   0%|          | 0/5 [00:00<?, ?it/s]

Processing videos for chiral ID: c1c62263 (5/16):   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing videos for chiral ID: ccff0ef2 (13/16):   0%|          | 0/5 [00:00<?, ?it/s]

Processing videos for chiral ID: edebebd4 (14/16):   0%|          | 0/2 [00:00<?, ?it/s]

Processing videos for chiral ID: 15c7cb08 (15/16):   0%|          | 0/3 [00:00<?, ?it/s]

Processing videos for chiral ID: 27f40159 (16/16):   0%|          | 0/1 [00:00<?, ?it/s]

286

In [59]:
correct = []
for key in answers:
    out = answers[key]
    try:
        ground_truth_index = out['ground_truth_index']
        predicted_index = int(out['answer'].replace('(', "").replace(")", "")) - 1
        correct.append(ground_truth_index == predicted_index)
    except:
        print(f"Failed to parse: {key}. Skipping.")
        display(Markdown(out['answer']))
        display(Markdown(f"**True:** {ground_truth_index + 1} {out['options'][ground_truth_index]}"))
        display(Markdown('---'))
        continue
correct = np.array(correct)
print("Number of parsed samples: ", len(correct))
accuracy = np.round(np.mean(correct) * 100., 2)
print("Accuracy: ", accuracy)

Failed to parse: /scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/44451.webm. Skipping.


The action shown in the video is a hand picking up several pens from a group of pens laid out on a surface. This is best described as taking something from a collection of similar items.

(1)

**True:** 1 Taking [one of many similar things on the table]

---

Failed to parse: /scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/157167.webm. Skipping.


...

Failed to parse: /scratch/shared/beegfs/piyush/datasets/SSv2/20bn-something-something-v2/122623.webm. Skipping.


The video shows a hand entering the frame from the bottom-left and moving towards a spinning coin located on the right side of the frame. The hand moves in a general direction from left to right to interact with the coin. As the hand approaches and makes contact (or gets very close), the coin stops spinning and settles.

Let's analyze the options:
(1) Pushing [something] from right to left: This would imply the object moves from the right side towards the left. While the coin's final resting position is slightly to the left of its initial spinning spot, the hand initiating the interaction is on the left, not the right. If the coin moved from right to left, the force would typically come from the right.

(2) Pushing [something] from left to right: This would imply the object moves from the left side towards the right, or that the force is applied from the left towards the right. The hand, which is the agent of the action, moves from the left side of the frame towards the right. If the hand were pushing, its action would be in the left-to-right direction.

Considering that the "action" refers to the movement and applied force of the hand, the hand's trajectory and point of contact are consistent with an action originating from the left and moving towards the right. Even if the coin's final small displacement is slightly to the left (possibly due to complex interaction of stopping a spin), the primary action of the hand itself is directed from left to right.

Therefore, describing the hand's action, the best match is pushing from left to right.

The final answer is $\boxed{2}$

**True:** 2 Pushing [something] from left to right

---

Number of parsed samples:  267
Accuracy:  89.14


In [60]:
(sum(correct) + 15) / len(answers)

np.float64(0.8846153846153846)